In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

In [ ]:
events = {
    "Dotcom-Blase": ("2001-02-01", "2003-06-30"),
    "Weltwirtschaftskrise": ("2008-01-01", "2009-04-30"),
    "Coronakrise": ("2019-09-01", "2020-04-30")
}

## Vergleich des Sentiments in Artikeln und Reden (LNTWS)

In [ ]:
artikel_m = pd.read_csv('artikel_sentiment_relevant_monat.csv')
artikel_q = pd.read_csv('artikel_sentiment_relevant_quartal.csv')
reden_m = pd.read_csv('reden_sentiment_relevant_monat.csv')
reden_q = pd.read_csv('reden_sentiment_relevant_quartal.csv')

Mittelwerte des Sentiments.

In [ ]:
mean_artikel_m = artikel_m['Sentiment Score'].mean()
mean_reden_m = reden_m['Sentiment Score'].mean()
mean_artikel_q = artikel_q['Sentiment Score'].mean()
mean_reden_q = reden_q['Sentiment Score'].mean()

print(f'Der Mittelwert der monatlich aggregierten Sentimentwerde in den Artikeln beträgt {mean_artikel_m}.')
print(f'Der Mittelwert der quartalsweise aggregierten Sentimentwerde in den Artikeln beträgt {mean_artikel_q}.')
print(f'Der Mittelwert der monatlich aggregierten Sentimentwerde in den Reden beträgt {mean_reden_m}.')
print(f'Der Mittelwert der quartalsweise aggregierten Sentimentwerde in den Reden beträgt {mean_reden_q}.')

Standardabweichung des Sentiments

In [ ]:
# Berechnung der Standardabweichung für die monatlichen und quartalsweisen Sentimentwerte
std_artikel_m = artikel_m['Sentiment Score'].std()
std_reden_m = reden_m['Sentiment Score'].std()
std_artikel_q = artikel_q['Sentiment Score'].std()
std_reden_q = reden_q['Sentiment Score'].std()

# Ausgabe der Standardabweichungen
print(f"Standardabweichung (Artikel, monatlich): {std_artikel_m}")
print(f"Standardabweichung (Reden, monatlich): {std_reden_m}")
print(f"Standardabweichung (Artikel, quartalsweise): {std_artikel_q}")
print(f"Standardabweichung (Reden, quartalsweise): {std_reden_q}")

Vergleich der Sentimententwicklungen aus Artikeln und Reden.

In [ ]:
# Konvertieren der Quartale und Monate in datetime-Format
artikel_q['Datum'] = pd.PeriodIndex(
    artikel_q['Jahr'].astype(str) + 'Q' + artikel_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

reden_q['Datum'] = pd.PeriodIndex(
    reden_q['Jahr'].astype(str) + 'Q' + reden_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

artikel_m['Datum'] = pd.to_datetime(artikel_m['Jahr'].astype(str) + '-' + artikel_m['Monat'].astype(str).str.zfill(2) + '-01')
reden_m['Datum'] = pd.to_datetime(reden_m['Jahr'].astype(str) + '-' + reden_m['Monat'].astype(str).str.zfill(2) + '-01')


# Begrenzen der Daten auf den Zeitraum bis Ende Q3 2021
end_date = pd.to_datetime("2021-09-30")

# Filtern der Sentiment-Daten
artikel_q = artikel_q[artikel_q['Datum'] <= end_date]
artikel_m = artikel_m[artikel_m['Datum'] <= end_date]
reden_q = reden_q[reden_q['Datum'] <= end_date]
reden_m = reden_m[reden_m['Datum'] <= end_date]

In [ ]:
# Erstellen des Plots
plt.figure(figsize=(12, 6))

# Plot für quartalsweises Sentiment
# plt.plot(artikel_q['Datum'], artikel_q['Sentiment Score'], label='LNTWS Sentiment der Artikel (quartalsweise)', color='blue')
# plt.plot(reden_q['Datum'], reden_q['Sentiment Score'], label='LNTWS Sentiment der Reden (quartalsweise)', color='darkgreen')

# Plot für monatliches Sentiment
plt.plot(artikel_m['Datum'], artikel_m['Sentiment Score'], label='LNTWS Sentiment der Artikel (monatlich)', color='cornflowerblue')
plt.plot(reden_m['Datum'], reden_m['Sentiment Score'], label='LNTWS Sentiment der Reden (monatlich)', color='darkseagreen')

# Horizontale Linie für Neutralität
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=artikel_q['Datum'].min(),
                           end=artikel_q['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Achsentitel und Beschriftungen
plt.title('', fontsize=14)
# plt.xlabel('Quartal', fontsize=12)
plt.ylabel('Sentiment-Score', fontsize=10)
plt.grid(alpha=0.3)
# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
plt.tight_layout()

# Speichern der Artikel
plt.savefig('vergleich_reden_artikel_relevant_m.png', format='png', dpi=600)

plt.show()

Sentimentwerte voneinander abziehen und plotten.

In [ ]:
# Zusammenführen der Tabellen anhand von Jahr und Monat
merged_df = pd.merge(artikel_m, reden_m, on=['Datum'], suffixes=('_artikel', '_reden'))

In [ ]:
# Berechnung der Differenz
merged_df['Sentiment Differenz'] = merged_df['Sentiment Score_artikel'] - merged_df['Sentiment Score_reden']

# Plot
plt.figure(figsize=(12, 6))
plt.plot(merged_df['Datum'], merged_df['Sentiment Differenz'], label='Sentiment-Differenz (Artikel - Reden)', color='midnightblue')

# Horizontale Linie für Neutralität
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=merged_df['Datum'].min(),
                           end=merged_df['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Achsentitel und Beschriftungen
plt.title('', fontsize=14)
# plt.xlabel('Quartal', fontsize=12)
plt.ylabel('Sentiment-Differenz', fontsize=10)
plt.grid(alpha=0.3)
# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel-reden_differenz_relevant_m.png', format='png', dpi=600)

# Plot anzeigen
plt.show()

Lag-Analyse durchführen.

In [ ]:
# Funktion der Lag-Analyse
def lag_analysis(data, max_lag, sentiment_column, gdp_column):
    lag_results = []
    for lag in range(-max_lag, max_lag + 1):  # Von negativem bis positivem Lag
        if lag < 0:  # Sentiment führt das BIP an
            lagged_sentiment = data[sentiment_column].shift(-lag)
            correlation = data[gdp_column].corr(lagged_sentiment)
        elif lag > 0:  # BIP führt das Sentiment an
            lagged_gdp = data[gdp_column].shift(lag)
            correlation = data[sentiment_column].corr(lagged_gdp)
        else:  # Kein Lag, normale Korrelation
            correlation = data[sentiment_column].corr(data[gdp_column])
        lag_results.append({'Lag': lag, 'Correlation': correlation})

    return pd.DataFrame(lag_results)

# Durchführen der Lag-Analyse
max_lag = 4  # Analysiere bis zu 4 Quartale vor/nach
lag_results = lag_analysis(merged_df, max_lag, 'Sentiment Score_artikel', 'Sentiment Score_reden')

# Zeigen der Ergebnisse
# print(lag_results)

# Plot der Lag-Analyse
plt.figure(figsize=(10, 6))
plt.plot(lag_results['Lag'], lag_results['Correlation'], marker='o', linestyle='-', color='midnightblue')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.title('Lag-Analyse: Korrelation zwischen den quartalsweisen Sentimentwerten', fontsize=14)
plt.xlabel('Lag (Monate)', fontsize=12)
plt.ylabel('Korrelation', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_reden_lag_relevant_q.png', format='png', dpi=600)

# Anzeigen des Plots
plt.show()

Korrelation mit BIP

In [ ]:
# Laden der Dateien
reden_q = pd.read_csv('reden_sentiment_relevant_quartal.csv')
artikel_q = pd.read_csv('artikel_sentiment_relevant_quartal.csv')
gdp_df = pd.read_csv('bip_veränderung.csv')

# Konvertieren der Quartale und Monate in datetime-Format
artikel_q['Datum'] = pd.PeriodIndex(
    artikel_q['Jahr'].astype(str) + 'Q' + artikel_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

reden_q['Datum'] = pd.PeriodIndex(
    reden_q['Jahr'].astype(str) + 'Q' + reden_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

# Erstellen des Quartalsformats für die BIP-Daten
gdp_df['Quarter'] = gdp_df['Quarter'].str.extract(r'(\d)').astype(int)  # Extrahiere die Zahl
gdp_df['Datum'] = pd.PeriodIndex(
    gdp_df['Year'].astype(str) + 'Q' + gdp_df['Quarter'].astype(str), freq='Q'
).to_timestamp()

# Reduzieren der DataFrames auf relevante Spalten
reden_q = reden_q[['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': 'Reden Score'})
artikel_q = artikel_q[['Datum', 'Sentiment Score']].rename(columns={'Sentiment Score': 'Artikel Score'})
gdp_df = gdp_df[['Datum', 'GDP Change %']]

# Merge der Datenframes anhand von Datum
merged_df = pd.merge(reden_q, artikel_q, on='Datum', how='inner')
merged_df = pd.merge(merged_df, gdp_df, on='Datum', how='inner')

# Überprüfen, ob der Merge korrekt war
print(merged_df.head())
print(merged_df.info())

# Konvertieren der Werte in numerische Typen
merged_df['Reden Score'] = pd.to_numeric(merged_df['Reden Score'], errors='coerce')
merged_df['Artikel Score'] = pd.to_numeric(merged_df['Artikel Score'], errors='coerce')
merged_df['BIP Veränderung %'] = pd.to_numeric(merged_df['GDP Change %'], errors='coerce')

# Entfernen fehlender Werte
numerical_df = merged_df.dropna()

# Überprüfen, ob fehlende Werte entfernt wurden
# print(numerical_df.info())

# Berechnen der Korrelation
correlation_matrix = numerical_df[['Reden Score', 'Artikel Score', 'BIP Veränderung %']].corr() # Only include relevant columns

# Heatmap erstellen
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", cbar=True)

# Titel und Achsentitel anpassen
# plt.title("Korrelation zwischen Sentiment- und BIP-Veränderung (quartalsweise)")
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_reden_bip_korrelation_relevant.png', format='png', dpi=600)

# Grafik anzeigen
plt.show()

Analyse in Krisenzeiten

In [ ]:
# Kombinieren aller Zeiträume in eine Liste und erweitere um zwei Quartale vor und nach den Ereignissen
crisis_periods = []
for start_date, end_date in events.values():
    start_date = pd.to_datetime(start_date) - pd.offsets.QuarterEnd(2)  # Zwei Quartale vor
    end_date = pd.to_datetime(end_date) + pd.offsets.QuarterEnd(2)     # Zwei Quartale nach
    crisis_periods.append((start_date, end_date))

# Filtern der Daten basierend auf den erweiterten Krisenzeiten
filtered_data = pd.DataFrame()

for start_date, end_date in crisis_periods:
    subset = numerical_df[(numerical_df['Datum'] >= start_date) & (numerical_df['Datum'] <= end_date)]
    filtered_data = pd.concat([filtered_data, subset])

# Funktion zur Berechnung von Korrelationen und p-Werten
def calculate_correlation_and_pvalues(df):
    corr_matrix = df.corr()
    pval_matrix = pd.DataFrame(index=df.columns, columns=df.columns)

    for col1 in df.columns:
        for col2 in df.columns:
            if col1 == col2:
                # Setzen der diagonalen p-Werte auf 0
                pval_matrix.loc[col1, col2] = 0
            else:
                # Berechnen der p-Werte
                _, p_value = pearsonr(df[col1], df[col2])
                pval_matrix.loc[col1, col2] = p_value

    return corr_matrix, pval_matrix

# Berechnung der Korrelationen und p-Werte für die gefilterten Daten
correlation_matrix_filtered, pvalue_matrix_filtered = calculate_correlation_and_pvalues(
    filtered_data[['Reden Score', 'Artikel Score', 'BIP Veränderung %']]
)

# Debugging: Überprüfen der Ergebnisse
print("Korrelationen während der Krisenzeiten (+/- 2 Quartale):\n", correlation_matrix_filtered)
print("\nP-Werte während der Krisenzeiten (+/- 2 Quartale):\n", pvalue_matrix_filtered)

# Heatmap erstellen
plt.figure(figsize=(10, 8))

# Korrelationen heatmap
sns.heatmap(correlation_matrix_filtered, annot=True, cmap="coolwarm", fmt=".2f", cbar=True)
plt.title("Korrelationen während der Krisenzeiten (+/- 2 Quartale)")
plt.tight_layout()
plt.savefig('korrelationen_krisen_2quartale.png', format='png', dpi=600)
plt.show()

# Heatmap für P-Werte
plt.figure(figsize=(10, 8))
sns.heatmap(pvalue_matrix_filtered.astype(float), annot=True, cmap="coolwarm", fmt=".2g", cbar=True)
plt.title("P-Werte während der Krisenzeiten (+/- 2 Quartale)")
plt.tight_layout()
plt.savefig('pvalues_krisen_2quartale.png', format='png', dpi=600)
plt.show()

Analyse außerhalb von Krisenzeiten

In [ ]:
# Funktion zur Berechnung von Korrelationen und p-Werten
def calculate_correlation_and_pvalues(df):
    corr_matrix = df.corr()
    pval_matrix = pd.DataFrame(index=df.columns, columns=df.columns)

    for col1 in df.columns:
        for col2 in df.columns:
            if col1 == col2:
                # Setze die diagonalen p-Werte auf 0
                pval_matrix.loc[col1, col2] = 0
            else:
                # Berechne p-Wert
                _, p_value = pearsonr(df[col1], df[col2])
                pval_matrix.loc[col1, col2] = p_value

    return corr_matrix, pval_matrix

# Erstellen einer Maske für die erweiterten Krisenzeiten
mask = pd.Series(False, index=numerical_df.index)

for start_date, end_date in crisis_periods:
    mask |= (numerical_df['Datum'] >= start_date) & (numerical_df['Datum'] <= end_date)

# Daten außerhalb der Krisenzeiten filtern
non_crisis_data = numerical_df[~mask]

# Überprüfen, ob Daten vorhanden sind
if not non_crisis_data.empty:
    # Berechne die Korrelationen und p-Werte für die nicht-Krisenzeiten
    correlation_matrix_non_crisis, pvalue_matrix_non_crisis = calculate_correlation_and_pvalues(
        non_crisis_data[['Reden Score', 'Artikel Score', 'BIP Veränderung %']]
    )

    # Debugging: Überprüfen der Ergebnisse
    print("Korrelationen außerhalb der Krisenzeiten:\n", correlation_matrix_non_crisis)
    print("\nP-Werte außerhalb der Krisenzeiten:\n", pvalue_matrix_non_crisis)

    # Heatmap der Korrelationen erstellen
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix_non_crisis, annot=True, cmap="coolwarm", fmt=".2f", cbar=True)
    plt.title("Korrelationen außerhalb der Krisenzeiten")
    plt.tight_layout()
    plt.savefig('artikel_reden_bip_korrelation_non_krisen_2_corr.png', format='png', dpi=600)
    plt.show()

    Heatmap der P-Werte erstellen
    plt.figure(figsize=(10, 8))
    sns.heatmap(pvalue_matrix_non_crisis.astype(float), annot=True, cmap="coolwarm", fmt=".2g", cbar=True)
    plt.title("P-Werte außerhalb der Krisenzeiten")
    plt.tight_layout()
    plt.savefig('artikel_reden_bip_korrelation_non_krisen_2_pvalues.png', format='png', dpi=600)
    plt.show()
else:
    print("Es sind keine Daten außerhalb der Krisenzeiten vorhanden.")

Korrelation über den gesamten Zeitraum

In [ ]:
# Funktion zur Berechnung von Korrelationen und p-Werten
def calculate_correlation_and_pvalues(df):
    # Spaltennamen
    cols = df.columns
    # Leere DataFrames für Korrelationen und p-Werte
    correlation_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    pvalue_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)

    # Berechnung paarweiser Korrelationen und p-Werte
    for col1 in cols:
        for col2 in cols:
            if col1 == col2:
                # Diagonale: Korrelation = 1, p-Wert = 0
                correlation_matrix.loc[col1, col2] = 1.0
                pvalue_matrix.loc[col1, col2] = 0.0
            else:
                # Berechnung der Korrelation und des p-Werts
                corr, pval = pearsonr(df[col1], df[col2])
                correlation_matrix.loc[col1, col2] = corr
                pvalue_matrix.loc[col1, col2] = pval

    return correlation_matrix, pvalue_matrix

# Beispiel: Berechnung für die numerischen Spalten
numerical_df = numerical_df[['Reden Score', 'Artikel Score', 'BIP Veränderung %']]
correlation_matrix, pvalue_matrix = calculate_correlation_and_pvalues(numerical_df)

# Debugging: Überprüfen der Ergebnisse
print("Korrelationen:\n", correlation_matrix)
print("P-Werte:\n", pvalue_matrix)

## Vergleich des LNTWS- und BPWS-Sentiments

In [ ]:
artikel_lntws_m = pd.read_csv('artikel_sentiment_relevant_monat.csv')
artikel_lntws_q = pd.read_csv('artikel_sentiment_relevant_quartal.csv')
artikel_bpws_m = pd.read_csv('artikel_bpw_relevant_monat.csv')
artikel_bpws_q = pd.read_csv('artikel_bpw_relevant_quartal.csv')

In [ ]:
artikel_bpws_q.head()

In [ ]:
# Zeitreihen konvertieren: Monatliche Daten
artikel_lntws_m['Datum'] = pd.to_datetime(
    artikel_lntws_m['Jahr'].astype(str) + '-' + artikel_lntws_m['Monat'].astype(str).str.zfill(2) + '-01'
)
artikel_bpws_m['Datum'] = pd.to_datetime(
    artikel_bpws_m['Jahr'].astype(str) + '-' + artikel_bpws_m['Monat'].astype(str).str.zfill(2) + '-01'
)

# Zeitreihen konvertieren: Quartalsweise Daten
artikel_lntws_q['Datum'] = pd.PeriodIndex(
    artikel_lntws_q['Jahr'].astype(str) + 'Q' + artikel_lntws_q['Quartal'].astype(str), freq='Q'
).to_timestamp()
artikel_bpws_q['Datum'] = pd.PeriodIndex(
    artikel_bpws_q['Jahr'].astype(str) + 'Q' + artikel_bpws_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

end_date = pd.to_datetime("2021-09-30")

# Filtern der Sentiment-Daten
artikel_lntws_q = artikel_lntws_q[artikel_lntws_q['Datum'] <= end_date]
artikel_lntws_m = artikel_lntws_m[artikel_lntws_m['Datum'] <= end_date]
artikel_bpws_q = artikel_bpws_q[artikel_bpws_q['Datum'] <= end_date]
artikel_bpws_m = artikel_bpws_m[artikel_bpws_m['Datum'] <= end_date]

# Mergen der monatlichen Daten
merged_monthly = pd.merge(
    artikel_lntws_m[['Datum', 'Sentiment Score']],
    artikel_bpws_m[['Datum', 'BPW Score']],
    on='Datum'
)

# Mergen der quartalsweisen Daten
merged_quarterly = pd.merge(
    artikel_lntws_q[['Datum', 'Sentiment Score']],
    artikel_bpws_q[['Datum', 'BPW Score']],
    on='Datum'
)

# Berechnung der Korrelationen
monthly_correlation = merged_monthly['Sentiment Score'].corr(merged_monthly['BPW Score'])
quarterly_correlation = merged_quarterly['Sentiment Score'].corr(merged_quarterly['BPW Score'])

# Ausgabe der Ergebnisse
print(f"Korrelation (monatlich) zwischen Sentiment-Score und BPW-Score: {monthly_correlation}")
print(f"Korrelation (quartalsweise) zwischen Sentiment-Score und BPW-Score: {quarterly_correlation}")

In [ ]:
# Plot der monatlichen Zeitreihen
plt.figure(figsize=(12, 6))
plt.plot(merged_monthly['Datum'], merged_monthly['Sentiment Score'], label='LNTWS Score', marker='o', linestyle='-', alpha=0.8)
plt.plot(merged_monthly['Datum'], merged_monthly['BPW Score'], label='BPWS Score', marker='s', linestyle='-', alpha=0.8)

# Beschriftungen und Titel
plt.title('Monatliche Zeitreihen: LNTWS Score und BPWS Score', fontsize=14)
plt.xlabel('Datum', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()

# Speichern des Plots (optional)
plt.savefig('monatliche_zeitreihen_plot.png', format='png', dpi=600)

# Plot anzeigen
plt.show()

In [ ]:
# Plot der quartalsweise Zeitreihen
plt.figure(figsize=(12, 6))
plt.plot(merged_quarterly['Datum'], merged_quarterly['Sentiment Score'], label='Sentiment Score', marker='o', linestyle='-', alpha=0.8)
plt.plot(merged_quarterly['Datum'], merged_quarterly['BPW Score'], label='BPW Score', marker='s', linestyle='-', alpha=0.8)

# Beschriftungen und Titel
plt.title('Quartalsweise Zeitreihen: Sentiment Score und BPW Score', fontsize=14)
plt.xlabel('Datum', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()

# Speichern des Plots (optional)
plt.savefig('quartalsweise_zeitreihen_plot.png', format='png', dpi=600)

# Plot anzeigen
plt.show()

## Vergleich des Sentiments in Artikeln und Reden (BPWS)

In [ ]:
artikel_bpws_m = pd.read_csv('artikel_bpw_relevant_monat.csv')
artikel_bpws_q = pd.read_csv('artikel_bpw_relevant_quartal.csv')
reden_bpws_m = pd.read_csv('reden_bpw_relevant_monat.csv')
reden_bpws_q = pd.read_csv('reden_bpw_relevant_quartal.csv')

Mittelwerte des Sentiments

In [ ]:
mean_artikel_m = artikel_bpws_m['BPW Score'].mean()
mean_reden_m = reden_bpws_m['BPW Score'].mean()
mean_artikel_q = artikel_bpws_q['BPW Score'].mean()
mean_reden_q = reden_bpws_q['BPW Score'].mean()

print(f'Der Mittelwert der monatlich aggregierten Sentimentwerte in den Artikeln beträgt {mean_artikel_m}.')
print(f'Der Mittelwert der quartalsweise aggregierten Sentimentwerte in den Artikeln beträgt {mean_artikel_q}.')
print(f'Der Mittelwert der monatlich aggregierten Sentimentwerte in den Reden beträgt {mean_reden_m}.')
print(f'Der Mittelwert der quartalsweise aggregierten Sentimentwerte in den Reden beträgt {mean_reden_q}.')

Standardabweichung des Sentiments

In [ ]:
# Berechnung der Standardabweichung für die monatlichen und quartalsweisen Sentimentwerte
std_artikel_m = artikel_bpws_m['BPW Score'].std()
std_reden_m = reden_bpws_m['BPW Score'].std()
std_artikel_q = artikel_bpws_q['BPW Score'].std()
std_reden_q = reden_bpws_q['BPW Score'].std()

# Ausgabe der Standardabweichungen
print(f"Standardabweichung (Artikel, monatlich): {std_artikel_m}")
print(f"Standardabweichung (Reden, monatlich): {std_reden_m}")
print(f"Standardabweichung (Artikel, quartalsweise): {std_artikel_q}")
print(f"Standardabweichung (Reden, quartalsweise): {std_reden_q}")

Grafischer Vergleich der Sentimententwicklung

In [ ]:
# Konvertieren der Quartale und Monate in datetime-Format
artikel_bpws_q['Datum'] = pd.PeriodIndex(
    artikel_bpws_q['Jahr'].astype(str) + 'Q' + artikel_bpws_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

reden_bpws_q['Datum'] = pd.PeriodIndex(
    reden_bpws_q['Jahr'].astype(str) + 'Q' + reden_bpws_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

artikel_bpws_m['Datum'] = pd.to_datetime(artikel_bpws_m['Jahr'].astype(str) + '-' + artikel_bpws_m['Monat'].astype(str).str.zfill(2) + '-01')
reden_bpws_m['Datum'] = pd.to_datetime(reden_bpws_m['Jahr'].astype(str) + '-' + reden_bpws_m['Monat'].astype(str).str.zfill(2) + '-01')

# Begrenzen der Daten auf den Zeitraum bis Ende Q3 2021
end_date = pd.to_datetime("2021-09-30")

# Filtern der Sentiment-Daten
artikel_bpws_q = artikel_bpws_q[artikel_bpws_q['Datum'] <= end_date]
artikel_bpws_m = artikel_bpws_m[artikel_bpws_m['Datum'] <= end_date]
reden_bpws_q = reden_bpws_q[reden_bpws_q['Datum'] <= end_date]
reden_bpws_m = reden_bpws_m[reden_bpws_m['Datum'] <= end_date]

In [ ]:
# Erstellen des Plots
plt.figure(figsize=(12, 6))

# Plot für quartalsweises Sentiment
plt.plot(artikel_bpws_q['Datum'], artikel_bpws_q['BPW Score'], label='BPWS Sentiment der Artikel (quartalsweise)', color='blue')
plt.plot(reden_bpws_q['Datum'], reden_bpws_q['BPW Score'], label='BPWS Sentiment der Reden (quartalsweise)', color='darkgreen')

# Plot für monatliches Sentiment
# plt.plot(artikel_bpws_m['Datum'], artikel_bpws_m['BPW Score'], label='BPWS Sentiment der Artikel (monatlich)', color='cornflowerblue')
# plt.plot(reden_bpws_m['Datum'], reden_bpws_m['BPW Score'], label='BPWS Sentiment der Reden (monatlich)', color='darkseagreen')

# Horizontale Linie für Neutralität
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=artikel_bpws_q['Datum'].min(),
                           end=artikel_bpws_q['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Achsentitel und Beschriftungen
plt.title('', fontsize=14)
# plt.xlabel('Quartal', fontsize=12)
plt.ylabel('Sentiment-Score', fontsize=10)
plt.grid(alpha=0.3)
# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
plt.tight_layout()

# Speichern der Artikel
plt.savefig('vergleich_reden_artikel_bpws_relevant_q.png', format='png', dpi=600)

plt.show()

Sentimentwerte voneinander abziehen

In [ ]:
# Zusammenführen der Tabellen anhand von Jahr und Monat
merged_bpws_df = pd.merge(artikel_bpws_m, reden_bpws_m, on=['Datum'], suffixes=('_artikel', '_reden'))

In [ ]:
# Berechnung der Differenz
merged_bpws_df['Sentiment Differenz'] = merged_bpws_df['BPW Score_artikel'] - merged_bpws_df['BPW Score_reden']

# Plot
plt.figure(figsize=(12, 6))
plt.plot(merged_bpws_df['Datum'], merged_bpws_df['Sentiment Differenz'], label='Sentiment-Differenz (Artikel - Reden)', color='midnightblue')

# Horizontale Linie für Neutralität
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=merged_bpws_df['Datum'].min(),
                           end=merged_bpws_df['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Achsentitel und Beschriftungen
plt.title('', fontsize=14)
# plt.xlabel('Quartal', fontsize=12)
plt.ylabel('Sentiment-Differenz', fontsize=10)
plt.grid(alpha=0.3)
# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel-reden_differenz_bpw_relevant_m.png', format='png', dpi=600)

# Plot anzeigen
plt.show()

In [ ]:
# Funktion für die Lag-Analyse
def lag_analysis(data, max_lag, sentiment_column, gdp_column):
    lag_results = []
    for lag in range(-max_lag, max_lag + 1):  # Von negativem bis positivem Lag
        if lag < 0:  # Sentiment führt das BIP an
            lagged_sentiment = data[sentiment_column].shift(-lag)
            correlation = data[gdp_column].corr(lagged_sentiment)
        elif lag > 0:  # BIP führt das Sentiment an
            lagged_gdp = data[gdp_column].shift(lag)
            correlation = data[sentiment_column].corr(lagged_gdp)
        else:  # Kein Lag, normale Korrelation
            correlation = data[sentiment_column].corr(data[gdp_column])
        lag_results.append({'Lag': lag, 'Correlation': correlation})

    return pd.DataFrame(lag_results)

# Durchführen der Lag-Analyse
max_lag = 4  # Analysiere bis zu 4 Quartale vor/nach
lag_results = lag_analysis(merged_bpws_df, max_lag, 'BPW Score_artikel', 'BPW Score_reden')

# Anzeigen die Ergebnisse
# print(lag_results)

# Plot der Lag-Analyse
plt.figure(figsize=(10, 6))
plt.plot(lag_results['Lag'], lag_results['Correlation'], marker='o', linestyle='-', color='midnightblue')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
# plt.title('Lag-Analyse: Korrelation zwischen den quartalsweisen Sentimentwerten', fontsize=14)
plt.xlabel('Lag (Monate)', fontsize=12)
plt.ylabel('Korrelation', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_reden_lag_bpw_relevant_q.png', format='png', dpi=600)

# Anzeigen des Plots
plt.show()

## Korrelation aller Datenreihen

### Korrelation seit 2005 mit ifo-Geschäftsklimaindex

In [ ]:
# Daten einlesen
reden_q = pd.read_csv('reden_sentiment_relevant_quartal.csv')
artikel_q = pd.read_csv('artikel_sentiment_relevant_quartal.csv')
gdp_df = pd.read_csv('bip_veränderung.csv')
bpw_artikel_q = pd.read_csv('artikel_bpw_relevant_quartal.csv')
bpw_reden_q = pd.read_csv('reden_bpw_relevant_quartal.csv')
ifo_data = pd.read_excel('ifo-Geschäftsklima.xlsx')

Korrelation

In [ ]:
# Zeitbereiche definieren
end_date = pd.to_datetime("2021-09-30")
start_date = pd.to_datetime("2005-01-01")

# Quartalsdaten für Artikel und Reden
for df, score_col, new_col in zip(
    [artikel_q, reden_q],
    ['Sentiment Score', 'Sentiment Score'],
    ['Artikel Score', 'Reden Score']
):
    df['Datum'] = pd.PeriodIndex(
        df['Jahr'].astype(str) + 'Q' + df['Quartal'].astype(str), freq='Q'
    ).to_timestamp()
    df.rename(columns={score_col: new_col}, inplace=True)

# Quartalsdaten für BPW-Sentiments (Artikel)
bpw_artikel_q['Datum'] = pd.PeriodIndex(
    bpw_artikel_q['Jahr'].astype(str) + 'Q' + bpw_artikel_q['Quartal'].astype(str), freq='Q'
).to_timestamp()
bpw_artikel_q.rename(columns={'BPW Sentiment': 'BPW Artikel'}, inplace=True)

# Quartalsdaten für BPW-Sentiments (Reden) aggregieren
bpw_reden_q = bpw_reden_q.rename(
    columns={'datum_jahr': 'Jahr', 'datum_quartal': 'Quartal'}
)
bpw_reden_q = bpw_reden_q.groupby(['Jahr', 'Quartal'], as_index=False)['BPW Score'].mean()
bpw_reden_q['Datum'] = pd.PeriodIndex(
    bpw_reden_q['Jahr'].astype(str) + 'Q' + bpw_reden_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

# BIP-Daten vorbereiten
gdp_df['Quarter'] = gdp_df['Quarter'].str.extract(r'(\d)').astype(int)
gdp_df['Datum'] = pd.PeriodIndex(
    gdp_df['Year'].astype(str) + 'Q' + gdp_df['Quarter'].astype(str), freq='Q'
).to_timestamp()
gdp_df.rename(columns={'GDP Change %': 'BIP Veränderung %'}, inplace=True)

# ifo-Geschäftsklimaindex vorbereiten
# Konvertieren der Spalte "Monat/Jahr" in datetime-Format
ifo_data['Datum'] = pd.to_datetime(ifo_data['Monat/Jahr'], format=' %m/%Y')
ifo_data['Quartal'] = ifo_data['Datum'].dt.to_period('Q')
ifo_quarterly = ifo_data.groupby('Quartal')['geschäftsklima-veränderung'].mean().reset_index()
ifo_quarterly['Datum'] = ifo_quarterly['Quartal'].dt.to_timestamp()
ifo_quarterly.rename(columns={'geschäftsklima-veränderung': 'ifo Index'}, inplace=True)

# Alle Daten auf den definierten Zeitraum begrenzen
datasets = [artikel_q, reden_q, bpw_artikel_q, bpw_reden_q, gdp_df, ifo_quarterly]
for df in datasets:
    df = df[(df['Datum'] >= start_date) & (df['Datum'] <= end_date)]

# BPW-Sentiments für Artikel und Reden vor dem Merge klar benennen
bpw_artikel_q.rename(columns={'BPW Score': 'BPWS Artikel'}, inplace=True)
bpw_reden_q.rename(columns={'BPW Score': 'BPWS Reden'}, inplace=True)

# Zusammenführen der Datensätze
merged_df = pd.merge(artikel_q[['Datum', 'Artikel Score']], reden_q[['Datum', 'Reden Score']], on='Datum', how='inner')
merged_df = pd.merge(merged_df, bpw_artikel_q[['Datum', 'BPWS Artikel']], on='Datum', how='inner')
merged_df = pd.merge(merged_df, bpw_reden_q[['Datum', 'BPWS Reden']], on='Datum', how='inner')
merged_df = pd.merge(merged_df, gdp_df[['Datum', 'BIP Veränderung %']], on='Datum', how='inner')
merged_df = pd.merge(merged_df, ifo_quarterly[['Datum', 'ifo Index']], on='Datum', how='inner')

# Spalten umbenennen, um die Korrelationen besser zuzuordnen
renamed_columns = {
    'Artikel Score': 'LNTWS Artikel',
    'Reden Score': 'LNTWS Reden',
    'BPWS Artikel': 'BPWS Artikel',
    'BPWS Reden': 'BPWS Reden',
    'ifo Index': 'ifo Geschäftsklima'
}
merged_df = merged_df.rename(columns=renamed_columns)

# Überprüfen, ob der Merge erfolgreich war
print(merged_df.head())
print(merged_df.info())

numerical_columns = merged_df.select_dtypes(include=['number'])

# Korrelationen berechnen
correlation_matrix = numerical_columns.corr()

# Heatmap der Korrelationen erstellen
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".3f", cbar=True)

# Titel und Layout
# plt.title("Korrelationen zwischen allen Datensätzen (quartalsweise)", fontsize=14)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('korrelationen_quartalsweise.png', format='png', dpi=600)

# Grafik anzeigen
plt.show()

p-Werte

In [ ]:
# Funktion zur Berechnung der Korrelation und der zugehörigen p-Werte
def calculate_p_values(df):
    """
    Berechnet p-Werte für alle Paarungen von numerischen Spalten in einem DataFrame.
    Gibt einen DataFrame mit den p-Werten zurück.
    """
    p_values = pd.DataFrame(index=df.columns, columns=df.columns)
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 != col2:
                _, p_value = pearsonr(df[col1], df[col2])  # Berechne p-Wert
                # Formatierung für kleine Werte
                if p_value < 1e-4:  # Wenn der p-Wert kleiner als 0.0001 ist
                    formatted_value = np.format_float_scientific(p_value, precision=2)  # Formatierung mit 2 Dezimalstellen
                else:
                    formatted_value = round(p_value, 4)  # Runde auf 4 Dezimalstellen
                p_values.loc[col1, col2] = formatted_value
            else:
                p_values.loc[col1, col2] = None  # Keine Selbstkorrelation
    return p_values

# p-Werte berechnen
p_values = calculate_p_values(numerical_columns)

# p-Werte als Tabelle ausgeben
print("p-Werte der Korrelationen:")
print(p_values)

# Optional: Speichern der p-Werte in einer CSV-Datei
p_values.to_csv("p_values_formatted.csv", sep=';', index=True, encoding='utf-8')
print("Die Matrix der p-Werte wurde in 'p_values_formatted.csv' gespeichert.")

### Korrelation seit 2000 ohne ifo-Geschäftsklimaindex

In [ ]:
# Daten einlesen
reden_q = pd.read_csv('reden_sentiment_relevant_quartal.csv')
artikel_q = pd.read_csv('artikel_sentiment_relevant_quartal.csv')
gdp_df = pd.read_csv('bip_veränderung.csv')
bpw_artikel_q = pd.read_csv('artikel_bpw_relevant_quartal.csv')
bpw_reden_q = pd.read_csv('reden_bpw_relevant_quartal.csv')

Korrelation

In [ ]:
# Zeitbereiche definieren
end_date = pd.to_datetime("2021-09-30")
start_date = pd.to_datetime("2000-01-01")

# Quartalsdaten für Artikel und Reden
for df, score_col, new_col in zip(
    [artikel_q, reden_q],
    ['Sentiment Score', 'Sentiment Score'],
    ['Artikel Score', 'Reden Score']
):
    df['Datum'] = pd.PeriodIndex(
        df['Jahr'].astype(str) + 'Q' + df['Quartal'].astype(str), freq='Q'
    ).to_timestamp()
    df.rename(columns={score_col: new_col}, inplace=True)

# Quartalsdaten für BPW-Sentiments (Artikel)
bpw_artikel_q['Datum'] = pd.PeriodIndex(
    bpw_artikel_q['Jahr'].astype(str) + 'Q' + bpw_artikel_q['Quartal'].astype(str), freq='Q'
).to_timestamp()
bpw_artikel_q.rename(columns={'BPW Sentiment': 'BPW Artikel'}, inplace=True)

# Quartalsdaten für BPW-Sentiments (Reden) aggregieren
bpw_reden_q = bpw_reden_q.rename(
    columns={'datum_jahr': 'Jahr', 'datum_quartal': 'Quartal'}
)
bpw_reden_q = bpw_reden_q.groupby(['Jahr', 'Quartal'], as_index=False)['BPW Score'].mean()
bpw_reden_q['Datum'] = pd.PeriodIndex(
    bpw_reden_q['Jahr'].astype(str) + 'Q' + bpw_reden_q['Quartal'].astype(str), freq='Q'
).to_timestamp()

# BIP-Daten vorbereiten
gdp_df['Quarter'] = gdp_df['Quarter'].str.extract(r'(\d)').astype(int)
gdp_df['Datum'] = pd.PeriodIndex(
    gdp_df['Year'].astype(str) + 'Q' + gdp_df['Quarter'].astype(str), freq='Q'
).to_timestamp()
gdp_df.rename(columns={'GDP Change %': 'BIP Veränderung %'}, inplace=True)

# Alle Daten auf den definierten Zeitraum begrenzen
datasets = [artikel_q, reden_q, bpw_artikel_q, bpw_reden_q, gdp_df]
for df in datasets:
    df = df[(df['Datum'] >= start_date) & (df['Datum'] <= end_date)]

# BPW-Sentiments für Artikel und Reden vor dem Merge klar benennen
bpw_artikel_q.rename(columns={'BPW Score': 'BPWS Artikel'}, inplace=True)
bpw_reden_q.rename(columns={'BPW Score': 'BPWS Reden'}, inplace=True)

# Zusammenführen der Datensätze mit klaren Namen
merged_df = pd.merge(artikel_q[['Datum', 'Artikel Score']], reden_q[['Datum', 'Reden Score']], on='Datum', how='inner')
merged_df = pd.merge(merged_df, bpw_artikel_q[['Datum', 'BPWS Artikel']], on='Datum', how='inner')
merged_df = pd.merge(merged_df, bpw_reden_q[['Datum', 'BPWS Reden']], on='Datum', how='inner')
merged_df = pd.merge(merged_df, gdp_df[['Datum', 'BIP Veränderung %']], on='Datum', how='inner')

# Spalten umbenennen, um die Korrelationen besser zuzuordnen
renamed_columns = {
    'Artikel Score': 'LNTWS Artikel',
    'Reden Score': 'LNTWS Reden',
    'BPWS Artikel': 'BPWS Artikel',
    'BPWS Reden': 'BPWS Reden'
}
merged_df = merged_df.rename(columns=renamed_columns)

# Überprüfen, ob der Merge erfolgreich war
print(merged_df.head())
print(merged_df.info())

numerical_columns = merged_df.select_dtypes(include=['number'])

# Korrelationen berechnen
correlation_matrix = numerical_columns.corr()

# Heatmap der Korrelationen erstellen
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".3f", cbar=True)

# Titel und Layout
# plt.title("Korrelationen zwischen allen Datensätzen (quartalsweise)", fontsize=14)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('korrelationen_quartalsweise.png', format='png', dpi=600)

# Grafik anzeigen
plt.show()

p-Werte

In [ ]:
# Funktion zur Berechnung der Korrelation und der zugehörigen p-Werte
def calculate_p_values(df):
    p_values = pd.DataFrame(index=df.columns, columns=df.columns)
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 != col2:
                _, p_value = pearsonr(df[col1], df[col2])  # Berechne p-Wert
                p_values.loc[col1, col2] = p_value
            else:
                p_values.loc[col1, col2] = None  # Keine Selbstkorrelation
    return p_values

# p-Werte berechnen
p_values = calculate_p_values(numerical_columns)

# p-Werte als Tabelle ausgeben
print("p-Werte der Korrelationen:")
print(p_values)